In [21]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

  Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl.metadata (3.5 kB)
Using cached tensorflow-2.16.1-cp311-cp311-win_amd64.whl (2.1 kB)


In [5]:
import cv2
import numpy as np
import os 
import matplotlib.pyplot as plt
import time 
import mediapipe as mp
import sklearn.model_selection as trainTestSplit
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [6]:
# load the mediapipe pose model
mpHolistic = mp.solutions.holistic
mpDrawing = mp.solutions.drawing_utils

In [7]:
def mpdetection(image, model):
    #convert the image to rgsd
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    #detect the pose and make predictions
    results = model.process(image)
    image.flags.writeable = True
    #convert that jawn back to bgr
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [8]:
def drawPose(image, results):
    #define drawing specifications
    landmarkDrawingSpec = mp.solutions.drawing_utils.DrawingSpec(color=(223, 27, 27), thickness=2, circle_radius=2)
    connectionDrawingSpec = mp.solutions.drawing_utils.DrawingSpec(color=(255, 255, 255), thickness=1, circle_radius=1)

    #draw landmarks and connections
    mp.solutions.drawing_utils.draw_landmarks(
        image, 
        results.left_hand_landmarks, 
        mp.solutions.holistic.HAND_CONNECTIONS, 
        landmarkDrawingSpec, 
        connectionDrawingSpec
    )
    mp.solutions.drawing_utils.draw_landmarks(
        image, 
        results.right_hand_landmarks, 
        mp.solutions.holistic.HAND_CONNECTIONS, 
        landmarkDrawingSpec, 
        connectionDrawingSpec
    )
    return image

In [59]:
capture = cv2.VideoCapture(0)

with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while capture.isOpened():
        ret, frame = capture.read()
        #detect the pose
        image, results = mpdetection(frame, holistic)
        #draw the pose
        drawPose(image, results)

        if ret:
            cv2.imshow('frame', image)
            #leave the camera
            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        else:
            print("Failed to grab frame")
            break

    capture.release()
    cv2.destroyAllWindows()


In [9]:
def extractKeypoints(results):
    #if results are detected, extract the landmarks
    if results.right_hand_landmarks:
        rightHand = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten()
    #otherwise, create zeros array of 21x3 
    else:
        rightHand = np.zeros(21*3)

    if results.left_hand_landmarks:
        leftHand = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten()
    else:
        leftHand = np.zeros(21*3)

    return np.concatenate([rightHand, leftHand])

In [10]:
dataPath = os.path.join('data')
#actions
actions = np.array(['swipe_left', 'swipe_right'])
#30 videos of each action/frame sets
numberSequence = 30
#30 frames in length per action respectively 
sequenceLength = 30 

In [52]:
for action in actions:
    for sequence in range(numberSequence):
        try:
            os.makedirs(os.path.join(dataPath, action, str(sequence)))
        except:
            pass

In [53]:
capture = cv2.VideoCapture(0)
with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(numberSequence):
            for frameNum in range(sequenceLength):
                #read the frame
                ret, frame = capture.read()
                #detect the pose
                image, results = mpdetection(frame, holistic)
                #draw the pose
                drawPose(image, results)
                #save the frame
                if frameNum == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} video number {}'.format(action, sequence), (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)


                keypoints = extractKeypoints(results)
                npyPath = os.path.join(dataPath, action, str(sequence), str(frameNum))
                np.save(npyPath, keypoints)
                #display the frame
                cv2.imshow('frame', image)
                #leave the camera
              
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    capture.release()
    cv2.destroyAllWindows()

In [11]:
labelMap = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in range(numberSequence):
        window = []
        for frameNum in range(sequenceLength):
            res = np.load(os.path.join(dataPath, action, str(sequence), '{}.npy'.format(frameNum)))
            window.append(res)
        sequences.append(window)
        labels.append(labelMap[action])

In [12]:
x = np.array(sequences)

In [13]:
y = to_categorical(np.array(labels)).astype(int)

In [14]:
xTrain, xTest, yTrain, yTest = trainTestSplit.train_test_split(x, y, test_size=0.05)

In [15]:
logDir = os.path.join('Logs')
tbCallback = TensorBoard(log_dir=logDir)

In [16]:
model=Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\luthi\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [17]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [18]:
model.fit(xTrain, yTrain, epochs=150, callbacks=[tbCallback])

Epoch 1/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - categorical_accuracy: 0.5267 - loss: 0.6888
Epoch 2/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.5267 - loss: 0.6732
Epoch 3/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.4746 - loss: 0.6606
Epoch 4/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.5384 - loss: 0.6222
Epoch 5/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.5514 - loss: 0.5924
Epoch 6/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.8673 - loss: 0.5451
Epoch 7/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.8660 - loss: 0.4754
Epoch 8/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - categorical_accuracy: 0.9545 - loss: 0.3721
Epoch 9/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - categorical_accuracy: 0.8856 - loss: 0.6388
Epoch 10/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - categorical_accuracy: 0.9220 - loss: 0.3440
Epoch 11/150
2/2 ━━━━━━━━━━━━━━━━━━━━ 0

In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        48,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 610,280 (2.33 MB)

 Trainable params: 203,426 (794.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 406,854 (1.55 MB)

In [21]:
res = model.predict(xTest)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


In [29]:
actions[np.argmax(res[2])]

'swipe_left'

In [27]:
actions[np.argmax(yTest[1])]

'swipe_left'

In [30]:
model.save('action.h5')

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [40]:
yhat = model.predict(xTrain)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [33]:
ytrue = np.argmax(yTest, axis=1).tolist()   
yhat = np.argmax(yhat, axis=1).tolist()

In [44]:
from scipy import stats

In [43]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [46]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

NameError: name 'image' is not defined

<Figure size 1800x1800 with 0 Axes>

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mpHolistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mpdetection(frame, holistic)
        print(results)
        
        # Draw landmarks
        drawPose(image, results)
        
        # 2. Prediction logic
        keypoints = extractKeypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()